In [8]:
import numpy as np
import pandas as pd

from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

from is_holiday import check_holiday
from utilites import create_hour_features

In [9]:
def encode(original_dataframe: pd.DataFrame, 
    feature_to_encode: str) -> pd.DataFrame:
    
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]], dtype=int)
    res = pd.concat([original_dataframe, dummies], axis=1)
    return res.drop(columns=feature_to_encode)

def load_and_preprocess(path: str) -> pd.DataFrame:
    df = pd.read_csv(path)
    # Drop unnecessary columns
    df = df[['timestamp', 'location_id', 'weather_condition', 'temperature', 'pedestrians_count']]
    
    # Rearrange date
    df['date'] = df.apply(lambda row: str(datetime.strptime(row['timestamp'], '%Y-%m-%dT%H:%M:%SZ')), axis = 1)
    df = df.drop(columns='timestamp')

    # One-hot encoding for weather
    df = encode(df, 'weather_condition')
    return df


In [10]:
path = 'data/hystreet_fussgaengerfrequenzen_seit2021.csv'
df = load_and_preprocess(path)
df

,location_id,temperature,pedestrians_count,date,weather_condition_clear-day,weather_condition_clear-night,weather_condition_cloudy,weather_condition_fog,weather_condition_partly-cloudy-day,weather_condition_partly-cloudy-night,weather_condition_rain,weather_condition_snow,weather_condition_wind
0,329,13.51,151,2021-09-28 22:00:00,0,0,1,0,0,0,0,0,0
1,331,13.51,118,2021-09-28 22:00:00,0,0,1,0,0,0,0,0,0
2,330,13.51,203,2021-09-28 22:00:00,0,0,1,0,0,0,0,0,0
3,329,13.46,19,2021-09-28 23:00:00,0,0,1,0,0,0,0,0,0
4,331,13.46,7,2021-09-28 23:00:00,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
63979,331,6.80,2053,2024-03-05 12:00:00,0,0,0,0,0,0,1,0,0
63980,330,6.80,1394,2024-03-05 12:00:00,0,0,0,0,0,0,1,0,0
63981,329,6.90,644,2024-03-05 13:00:00,0,0,0,0,0,0,1,0,0
63982,331,6.90,477,2024-03-05 13:00:00,0,0,0,0,0,0,1,0,0


In [11]:
df_mid = df.loc[df['location_id'] == 330]
df_mid = df.drop(columns='location_id')

# Temporary drop date
df_mid['holiday'] = df.apply(lambda row: check_holiday(row['date']), axis = 1)
df_mid = df_mid.drop(columns='date')
# df_mid['date'][0].split()
# 

In [12]:
train, test = train_test_split(df_mid, test_size=0.2)

In [13]:
forest = RandomForestRegressor()
forest.fit(train.drop(columns='pedestrians_count'), train['pedestrians_count'])

RandomForestRegressor()

In [14]:
forest.score(test.drop(columns='pedestrians_count'), test['pedestrians_count'])

0.24424248281020866

In [ ]:
a = '2024-08-01 15:30:00'
a.split()[0]